In [1]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

#X_train = X_train - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]
#X_test = X_test - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]

X_train_full, y_train_full = X_train, y_train

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]



c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\ferbi\appd

In [2]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import math

def execute_code(rand_seed = 1, lambda_exp = -6.0):
    LOC = """

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

print("Now:", now)

n_inputs = X_train.shape[1]
n_hidden1 = 1024
n_hidden2 = 1024
n_hidden3 = 1024
n_hidden4 = 1024
n_hidden5 = 1024
n_outputs = 10

def reset_graph(seed=43):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph(@rand_seed@)

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

epsVal = pow(10.0, -8.0)

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs + n_neurons)
        init = tf.random_normal((n_inputs, n_neurons), mean=0.0, stddev=stddev)
        
        W = tf.Variable(init, name="kernel")
        #W = tf.cast(tf.abs(W) >= epsVal, W.dtype) * W
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.add(tf.matmul(X, W), b)
                
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    hidden3 = neuron_layer(hidden2, n_hidden3, name="hidden3", activation=tf.nn.relu)
    hidden4 = neuron_layer(hidden3, n_hidden4, name="hidden4", activation=tf.nn.relu)
    hidden5 = neuron_layer(hidden4, n_hidden5, name="hidden5", activation=tf.nn.relu)
    logits = neuron_layer(hidden5, n_outputs, name="outputs")

hidden1_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden1/kernel:0")
hidden2_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden2/kernel:0")
hidden3_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden3/kernel:0")
hidden4_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden4/kernel:0")
hidden5_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden5/kernel:0")
logits_W = tf.get_default_graph().get_tensor_by_name("dnn/outputs/kernel:0")
    
def calcRegValue(W, type=1, eps=0.00001):
    if type == 1:
        wMax = 1.0
        gt_eps = tf.minimum(1.0, tf.add(1.0, tf.constant(eps/(wMax-eps))*tf.subtract(tf.abs(W), wMax)))
        #gt_eps = 1.0  
        #lt_eps = tf.constant((1-eps)/eps)*tf.abs(W)
        lt_eps = 0.0
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*gt_eps + tf.cast(tf.abs(W) < eps, W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 2:
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*W
        return tf.reduce_sum(reg_value)
    elif type == 3:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.pow(W, 2))))
    elif type == 4:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.abs(W))))
    elif type == 5:
        return tf.reduce_sum(tf.abs(W))
    elif type == 6:
        return tf.reduce_sum(tf.pow(W, 2))
    elif type == 7:
        gt_eps = 1.0  
        epsMat = tf.constant(eps, shape=W.get_shape())
        lt_eps = 1.0 - tf.pow(epsMat, tf.abs(W))
        reg_value = tf.cast(tf.abs(W) >= np.sqrt(eps), W.dtype)*gt_eps + tf.cast(tf.abs(W) < np.sqrt(eps), W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 8:
        eMat = tf.constant(math.e, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(eMat, tf.subtract(0.0, 500*tf.pow(W, 2)))))
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    regType = 8
    total_regvalue = tf.add_n([calcRegValue(hidden1_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden2_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden3_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden4_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden5_W, type=regType, eps=epsVal), \
                               calcRegValue(logits_W, type=regType, eps=epsVal)])
    
    total_nzw = tf.cast(tf.add_n([tf.count_nonzero(hidden1_W), \
                                  tf.count_nonzero(hidden2_W), \
                                  tf.count_nonzero(hidden3_W), \
                                  tf.count_nonzero(hidden4_W), \
                                  tf.count_nonzero(hidden5_W), \
                                  tf.count_nonzero(logits_W)]), tf.float32)
    total_nzw_eps = tf.cast(tf.add_n([tf.reduce_sum(tf.cast(abs(hidden1_W) < epsVal, hidden1_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden2_W) < epsVal, hidden2_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden3_W) < epsVal, hidden3_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden4_W) < epsVal, hidden4_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden5_W) < epsVal, hidden5_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(logits_W) < epsVal, logits_W.dtype))]), tf.int64)
    
    lambdaParam = tf.pow(10.0, @lambda_exp@, name="lambdaParam")
    loss_total = tf.add(tf.reduce_mean(xentropy, name="loss_total"), tf.scalar_mul(lambdaParam, total_regvalue))
    #loss_total = tf.reduce_mean(xentropy, name="loss_total")
    
learning_rate = tf.placeholder(tf.float32, shape=(), name="learning_rate")
with tf.name_scope("train"):
    #optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    training_op = optimizer.minimize(loss_total)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

reg_summary = tf.summary.scalar('Count_Reg', total_regvalue)
loss_summary = tf.summary.scalar('Loss_Total', loss_total)
err_summary = tf.summary.scalar('Class_Err_Perc', (1-accuracy)*100)
#file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 250
batch_size = 20

def shuffle_batch(X, y, batch_size):
    np.random.seed(43)
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    batch_num = -1
    for batch_idx in np.array_split(rnd_idx, n_batches):
        batch_num = batch_num + 1
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield batch_num, X_batch, y_batch

def get_lr_step(epochCounter, start_lr, stepsize, end_lr, lr_iter_period):
    lr = start_lr - math.floor(epochCounter/lr_iter_period) * stepsize
    return max(lr, end_lr)
        
def get_lr_triangle(epochCounter, lr_min, lr_max, lr_iter_period):
    cycle = math.floor(1 + epochCounter/(2*lr_iter_period))
    x = abs(epochCounter/lr_iter_period - 2*cycle + 1)
    lr = lr_min + (lr_max - lr_min) * max(0, (1-x)/math.pow(2, cycle-1))
    return lr

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

start_time = datetime.now()

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        lr = get_lr_triangle(epoch, lr_min=0.01, lr_max=0.25, lr_iter_period=20)
        #lr=0.01
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, learning_rate: lr})
        
        loss_val, reg_loss_val, lambda1, acc_val = sess.run([loss_total, total_regvalue, lambdaParam, accuracy], \
                                                   feed_dict={X: X_valid, y: y_valid, learning_rate: lr})
        base_loss_val = loss_val - lambda1*reg_loss_val
        
        if best_loss > loss_val:
            save_path = saver.save(sess, "./models/model_" + now + ".ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation losses: {:.4f}, {:.4f}, {:.4f}\tBest Loss: {:.4f} ({})\tAccuracy: {:.2f}%".format(
            epoch, base_loss_val, lambda1*reg_loss_val, loss_val, best_loss, checks_without_progress, acc_val * 100))

end_time = datetime.now()
running_time = end_time - start_time
print("Total running time:\t", running_time.seconds)

with tf.Session() as sess:
    saver.restore(sess, "./models/model_" + now + ".ckpt") # or better, use save_path
    #saver.restore(sess, "./models/model_20190129210937.ckpt") # or better, use save_path
    n_batches = len(X_test) // 2000
    batch_remaining = n_batches
    correct_count = 0.0
    for batch_idx in np.array_split(range(len(X_test)), n_batches):
        correct_count = correct_count + len(batch_idx)*accuracy.eval(feed_dict={X: X_test[batch_idx], y: y_test[batch_idx]})
        print("Batches remaining:    ", batch_remaining)
        batch_remaining = batch_remaining - 1
print("Test error rate:    {:.10f}%".format(100*(1-correct_count/len(X_test)))) 
"""
    LOC = LOC.replace("@rand_seed@", str(rand_seed))
    LOC = LOC.replace("@lambda_exp@", str(lambda_exp))
    exec(LOC)

In [3]:
for j in [-4.0, -4.2, -4.4, -4.6, -4.8, -5.0, -5.2, -5.4, -5.6, -5.8, -6.0, -6.2, -6.4, -6.6, -6.8, -7.0]:
    for i in [1, 2, 3]:
        print("RANDOM SEED:  ", i, "\tLAMBDA_EXP:  ", j)
        execute_code(rand_seed=i, lambda_exp=j)

RANDOM SEED:   1 	LAMBDA_EXP:   -4.0
Now: 20240304204224
0	Validation losses: 0.5780, 65.7579, 66.3358	Best Loss: 66.3358 (0)	Accuracy: 77.36%
1	Validation losses: 0.5117, 24.1462, 24.6580	Best Loss: 24.6580 (0)	Accuracy: 83.22%
2	Validation losses: 0.5091, 14.2854, 14.7945	Best Loss: 14.7945 (0)	Accuracy: 82.00%
3	Validation losses: 0.5279, 10.3116, 10.8395	Best Loss: 10.8395 (0)	Accuracy: 80.96%
4	Validation losses: 0.5057, 8.2546, 8.7603	Best Loss: 8.7603 (0)	Accuracy: 82.32%
5	Validation losses: 0.5786, 7.0449, 7.6234	Best Loss: 7.6234 (0)	Accuracy: 78.84%
6	Validation losses: 0.4682, 6.2286, 6.6968	Best Loss: 6.6968 (0)	Accuracy: 83.74%
7	Validation losses: 0.4733, 5.6517, 6.1249	Best Loss: 6.1249 (0)	Accuracy: 82.50%
8	Validation losses: 0.4755, 5.2412, 5.7167	Best Loss: 5.7167 (0)	Accuracy: 83.06%
9	Validation losses: 0.5326, 4.9089, 5.4415	Best Loss: 5.4415 (0)	Accuracy: 81.60%
10	Validation losses: 0.4695, 4.6361, 5.1055	Best Loss: 5.1055 (0)	Accuracy: 83.72%
11	Validation los

19	Validation losses: 0.4969, 3.6373, 4.1342	Best Loss: 4.1342 (0)	Accuracy: 82.30%
20	Validation losses: nan, nan, nan	Best Loss: 4.1342 (1)	Accuracy: 0.00%
21	Validation losses: nan, nan, nan	Best Loss: 4.1342 (2)	Accuracy: 0.00%
22	Validation losses: nan, nan, nan	Best Loss: 4.1342 (3)	Accuracy: 0.00%
23	Validation losses: nan, nan, nan	Best Loss: 4.1342 (4)	Accuracy: 0.00%
24	Validation losses: nan, nan, nan	Best Loss: 4.1342 (5)	Accuracy: 0.00%
25	Validation losses: nan, nan, nan	Best Loss: 4.1342 (6)	Accuracy: 0.00%
26	Validation losses: nan, nan, nan	Best Loss: 4.1342 (7)	Accuracy: 0.00%
27	Validation losses: nan, nan, nan	Best Loss: 4.1342 (8)	Accuracy: 0.00%
28	Validation losses: nan, nan, nan	Best Loss: 4.1342 (9)	Accuracy: 0.00%
29	Validation losses: nan, nan, nan	Best Loss: 4.1342 (10)	Accuracy: 0.00%
30	Validation losses: nan, nan, nan	Best Loss: 4.1342 (11)	Accuracy: 0.00%
31	Validation losses: nan, nan, nan	Best Loss: 4.1342 (12)	Accuracy: 0.00%
32	Validation losses: nan

33	Validation losses: nan, nan, nan	Best Loss: 3.3443 (17)	Accuracy: 0.00%
34	Validation losses: nan, nan, nan	Best Loss: 3.3443 (18)	Accuracy: 0.00%
35	Validation losses: nan, nan, nan	Best Loss: 3.3443 (19)	Accuracy: 0.00%
36	Validation losses: nan, nan, nan	Best Loss: 3.3443 (20)	Accuracy: 0.00%
Early stopping!
Total running time:	 2985
INFO:tensorflow:Restoring parameters from ./models/model_20240305001255.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    17.4899995327%
RANDOM SEED:   3 	LAMBDA_EXP:   -4.2
Now: 20240305010242
0	Validation losses: 0.5042, 60.7106, 61.2149	Best Loss: 61.2149 (0)	Accuracy: 83.68%
1	Validation losses: 0.4779, 22.2488, 22.7267	Best Loss: 22.7267 (0)	Accuracy: 83.14%
2	Validation losses: 0.4663, 12.5549, 13.0213	Best Loss: 13.0213 (0)	Accuracy: 83.64%
3	Validation losses: 0.5246, 8.7077, 9.2323	Best Loss: 9.2323 (0)	Accuracy: 80.74%
4	Validation losses: 0.

24	Validation losses: nan, nan, nan	Best Loss: 3.1736 (13)	Accuracy: 0.00%
25	Validation losses: nan, nan, nan	Best Loss: 3.1736 (14)	Accuracy: 0.00%
26	Validation losses: nan, nan, nan	Best Loss: 3.1736 (15)	Accuracy: 0.00%
27	Validation losses: nan, nan, nan	Best Loss: 3.1736 (16)	Accuracy: 0.00%
28	Validation losses: nan, nan, nan	Best Loss: 3.1736 (17)	Accuracy: 0.00%
29	Validation losses: nan, nan, nan	Best Loss: 3.1736 (18)	Accuracy: 0.00%
30	Validation losses: nan, nan, nan	Best Loss: 3.1736 (19)	Accuracy: 0.00%
31	Validation losses: nan, nan, nan	Best Loss: 3.1736 (20)	Accuracy: 0.00%
Early stopping!
Total running time:	 2571
INFO:tensorflow:Restoring parameters from ./models/model_20240305022843.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    20.7700002193%
RANDOM SEED:   3 	LAMBDA_EXP:   -4.4
Now: 20240305031135
0	Validation losses: 0.4737, 51.0482, 51.5218	Best Loss: 51.5218

7	Validation losses: 0.4483, 2.9137, 3.3620	Best Loss: 3.3620 (0)	Accuracy: 84.74%
8	Validation losses: 0.3955, 2.5995, 2.9950	Best Loss: 2.9950 (0)	Accuracy: 85.34%
9	Validation losses: 0.4561, 2.3854, 2.8414	Best Loss: 2.8414 (0)	Accuracy: 84.48%
10	Validation losses: 0.4362, 2.2031, 2.6393	Best Loss: 2.6393 (0)	Accuracy: 83.82%
11	Validation losses: 0.5528, 2.0712, 2.6241	Best Loss: 2.6241 (0)	Accuracy: 80.88%
12	Validation losses: 0.5222, 1.9987, 2.5209	Best Loss: 2.5209 (0)	Accuracy: 81.68%
13	Validation losses: 0.4410, 1.8879, 2.3289	Best Loss: 2.3289 (0)	Accuracy: 84.22%
14	Validation losses: 0.4675, 1.8098, 2.2773	Best Loss: 2.2773 (0)	Accuracy: 83.32%
15	Validation losses: 0.3966, 1.7519, 2.1485	Best Loss: 2.1485 (0)	Accuracy: 86.30%
16	Validation losses: 0.4290, 1.6874, 2.1164	Best Loss: 2.1164 (0)	Accuracy: 83.54%
17	Validation losses: 0.4475, 1.6325, 2.0799	Best Loss: 2.0799 (0)	Accuracy: 84.20%
18	Validation losses: 1.1094, 1.7811, 2.8905	Best Loss: 2.0799 (1)	Accuracy: 65

...

13	Validation losses: 0.3076, 0.5227, 0.8303	Best Loss: 0.8303 (0)	Accuracy: 88.40%
14	Validation losses: 0.3074, 0.4936, 0.8010	Best Loss: 0.8010 (0)	Accuracy: 88.38%
15	Validation losses: 0.3052, 0.4664, 0.7716	Best Loss: 0.7716 (0)	Accuracy: 89.30%
16	Validation losses: 0.3244, 0.4423, 0.7667	Best Loss: 0.7667 (0)	Accuracy: 88.50%
17	Validation losses: 0.2948, 0.4189, 0.7138	Best Loss: 0.7138 (0)	Accuracy: 89.56%
18	Validation losses: 0.3029, 0.3973, 0.7002	Best Loss: 0.7002 (0)	Accuracy: 88.88%
19	Validation losses: 0.3652, 0.3787, 0.7439	Best Loss: 0.7002 (1)	Accuracy: 88.54%
20	Validation losses: 0.3523, 0.3632, 0.7156	Best Loss: 0.7002 (2)	Accuracy: 87.44%
21	Validation losses: 0.3365, 0.3479, 0.6844	Best Loss: 0.6844 (0)	Accuracy: 88.26%
22	Validation losses: 0.3418, 0.3331, 0.6749	Best Loss: 0.6749 (0)	Accuracy: 88.50%
23	Validation losses: 0.3390, 0.3201, 0.6590	Best Loss: 0.6590 (0)	Accuracy: 88.56%
24	Validation losses: 0.3308, 0.3090, 0.6398	Best Loss: 0.6398 (0)	Accuracy:

8	Validation losses: 0.3027, 0.4704, 0.7731	Best Loss: 0.7731 (0)	Accuracy: 88.80%
9	Validation losses: 0.3429, 0.4579, 0.8009	Best Loss: 0.7731 (1)	Accuracy: 88.32%
10	Validation losses: 0.3144, 0.4446, 0.7590	Best Loss: 0.7590 (0)	Accuracy: 88.56%
11	Validation losses: 0.3226, 0.4312, 0.7538	Best Loss: 0.7538 (0)	Accuracy: 88.04%
12	Validation losses: 0.3402, 0.4170, 0.7573	Best Loss: 0.7538 (1)	Accuracy: 88.34%
13	Validation losses: 0.3462, 0.4032, 0.7494	Best Loss: 0.7494 (0)	Accuracy: 88.12%
14	Validation losses: 0.3334, 0.3892, 0.7226	Best Loss: 0.7226 (0)	Accuracy: 88.66%
15	Validation losses: 0.3407, 0.3758, 0.7165	Best Loss: 0.7165 (0)	Accuracy: 87.96%
16	Validation losses: 0.3052, 0.3624, 0.6675	Best Loss: 0.6675 (0)	Accuracy: 89.38%
17	Validation losses: 0.3108, 0.3498, 0.6606	Best Loss: 0.6606 (0)	Accuracy: 89.38%
18	Validation losses: 0.3402, 0.3379, 0.6781	Best Loss: 0.6606 (1)	Accuracy: 88.92%
19	Validation losses: 0.3462, 0.3269, 0.6731	Best Loss: 0.6606 (2)	Accuracy: 8

6	Validation losses: 0.2886, 0.3216, 0.6103	Best Loss: 0.6103 (0)	Accuracy: 89.24%
7	Validation losses: 0.3253, 0.3179, 0.6432	Best Loss: 0.6103 (1)	Accuracy: 87.96%
8	Validation losses: 0.3098, 0.3137, 0.6235	Best Loss: 0.6103 (2)	Accuracy: 88.70%
9	Validation losses: 0.3332, 0.3093, 0.6425	Best Loss: 0.6103 (3)	Accuracy: 88.42%
10	Validation losses: 0.3004, 0.3045, 0.6049	Best Loss: 0.6049 (0)	Accuracy: 89.38%
11	Validation losses: 0.3128, 0.2994, 0.6122	Best Loss: 0.6049 (1)	Accuracy: 88.66%
12	Validation losses: 0.3858, 0.2941, 0.6799	Best Loss: 0.6049 (2)	Accuracy: 87.04%
13	Validation losses: 0.3215, 0.2887, 0.6103	Best Loss: 0.6049 (3)	Accuracy: 88.10%
14	Validation losses: 0.2971, 0.2833, 0.5803	Best Loss: 0.5803 (0)	Accuracy: 89.28%
15	Validation losses: 0.3602, 0.2782, 0.6384	Best Loss: 0.5803 (1)	Accuracy: 88.12%
16	Validation losses: 0.3057, 0.2728, 0.5785	Best Loss: 0.5785 (0)	Accuracy: 89.50%
17	Validation losses: 0.3201, 0.2672, 0.5873	Best Loss: 0.5785 (1)	Accuracy: 89.

18	Validation losses: 0.3320, 0.2616, 0.5936	Best Loss: 0.5824 (4)	Accuracy: 88.60%
19	Validation losses: 0.3300, 0.2567, 0.5866	Best Loss: 0.5824 (5)	Accuracy: 88.48%
20	Validation losses: 0.3775, 0.2519, 0.6294	Best Loss: 0.5824 (6)	Accuracy: 87.62%
21	Validation losses: nan, nan, nan	Best Loss: 0.5824 (7)	Accuracy: 0.00%
22	Validation losses: nan, nan, nan	Best Loss: 0.5824 (8)	Accuracy: 0.00%
23	Validation losses: nan, nan, nan	Best Loss: 0.5824 (9)	Accuracy: 0.00%
24	Validation losses: nan, nan, nan	Best Loss: 0.5824 (10)	Accuracy: 0.00%
25	Validation losses: nan, nan, nan	Best Loss: 0.5824 (11)	Accuracy: 0.00%
26	Validation losses: nan, nan, nan	Best Loss: 0.5824 (12)	Accuracy: 0.00%
27	Validation losses: nan, nan, nan	Best Loss: 0.5824 (13)	Accuracy: 0.00%
28	Validation losses: nan, nan, nan	Best Loss: 0.5824 (14)	Accuracy: 0.00%
29	Validation losses: nan, nan, nan	Best Loss: 0.5824 (15)	Accuracy: 0.00%
30	Validation losses: nan, nan, nan	Best Loss: 0.5824 (16)	Accuracy: 0.00%
3

31	Validation losses: 0.4431, 0.1648, 0.6079	Best Loss: 0.4966 (15)	Accuracy: 90.48%
32	Validation losses: 0.4421, 0.1636, 0.6056	Best Loss: 0.4966 (16)	Accuracy: 90.40%
33	Validation losses: 0.5010, 0.1624, 0.6634	Best Loss: 0.4966 (17)	Accuracy: 90.56%
34	Validation losses: 0.5338, 0.1614, 0.6952	Best Loss: 0.4966 (18)	Accuracy: 90.38%
35	Validation losses: 0.5720, 0.1604, 0.7325	Best Loss: 0.4966 (19)	Accuracy: 90.48%
36	Validation losses: 0.6126, 0.1597, 0.7723	Best Loss: 0.4966 (20)	Accuracy: 90.68%
Early stopping!
Total running time:	 3138
INFO:tensorflow:Restoring parameters from ./models/model_20240306201204.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    11.1300003529%
RANDOM SEED:   3 	LAMBDA_EXP:   -7.0
Now: 20240306210424
0	Validation losses: 0.4283, 0.2113, 0.6396	Best Loss: 0.6396 (0)	Accuracy: 85.32%
1	Validation losses: 0.3540, 0.2110, 0.5650	Best Loss: 0.5650 (0)	Accur